In [1]:
%run C:/Users/MohammedSB/Desktop/projects/Hypertension/Requirements.ipynb

In [2]:
# Seed
set_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
PATH = r"D:\\MohammedSB\\Fundus\\"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = r"C:\Users\MohammedSB\Desktop\projects\Hypertension\models"

In [5]:
train_set, test_set = get_datasets()

In [6]:
BATCH_SIZE = 8

transform = T.Compose([
    T.Resize((500, 500)),
    T.ToTensor(),
    T.Normalize(mean=[0.3675, 0.1760, 0.1047], std=[0.2289, 0.1180, 0.0748]),
])

train_dataset = HypertensionDataset(CSV_PATH, train_set, train_transform=transform)
test_dataset = HypertensionDataset(CSV_PATH, test_set, test_transform=transform)

In [7]:
def calculate_results(N, trues, probs, threshold=0.5):
    
    f1_scores = []
    auc_scores = []
    pr_scores = []
    precision_scores = []
    recall_scores = []
    specificity_scores = []

    for i in range(N):
        fpr, tpr, _ = roc_curve(trues[i], probs[i])
        precision, recall, _ = precision_recall_curve(trues[i], probs[i])

        f1_scores.append(f1_score(trues[i], np.array(probs[i])>=threshold))
        auc_scores.append(auc(fpr, tpr))
        pr_scores.append(auc(recall, precision))
        precision_scores.append(precision_score(trues[i], np.array(probs[i])>=threshold))
        recall_scores.append(recall_score(trues[i], np.array(probs[i])>=threshold))
        specificity_scores.append(recall_score(trues[i], np.array(probs[i])>=threshold, pos_label=0))
    
    print(f"f1: {[round(s, 3) for s in np.percentile(f1_scores, [2.5, 97.5])]}",
            f"average: {round(np.percentile(f1_scores, [2.5, 97.5]).mean(), 3)}") 
    print(f"auroc: {[round(s, 3) for s in np.percentile(auc_scores, [2.5, 97.5])]}",
            f"average: {round(np.percentile(auc_scores, [2.5, 97.5]).mean(), 3)}") 
    print(f"auprc: {[round(s, 3) for s in np.percentile(pr_scores, [2.5, 97.5])]}",
            f"average: {round(np.percentile(pr_scores, [2.5, 97.5]).mean(), 3)}") 
    print(f"precision: {[round(s, 3) for s in np.percentile(precision_scores, [2.5, 97.5])]}",
            f"average: {round(np.percentile(precision_scores, [2.5, 97.5]).mean(), 3)}") 
    print(f"recall: {[round(s, 3) for s in np.percentile(recall_scores, [2.5, 97.5])]}",
            f"average: {round(np.percentile(recall_scores, [2.5, 97.5]).mean(), 3)}")
    print(f"specificity: {[round(s, 3) for s in np.percentile(specificity_scores, [2.5, 97.5])]}",
            f"average: {round(np.percentile(specificity_scores, [2.5, 97.5]).mean(), 3)}")
    
    
def bootstrap(N, models, data_set, device, criterion=None, method="torch"):
    num_samples = len(data_set)
    trues, probs = dict(), dict()

    for i in tqdm(range(N)):

        if method == "torch":
            sampler = torch.utils.data.RandomSampler(data_set, replacement=True, num_samples=num_samples)
            data_loader = DataLoader(data_set, batch_size=8, sampler=sampler)

            metrics = test(models=models, criterion=criterion, test_loader=data_loader, device=device, show_output=False)
            y_true = metrics["y_true"].squeeze(1).numpy()
            y_prob = metrics["y_prob"].squeeze(1).numpy()

            trues[i] = y_true.tolist()
            probs[i] = y_prob.tolist()
        elif method == "sklearn":
            bootstrapped_data = resample(data_set, replace=True, n_samples=num_samples)
            x, y = bootstrapped_data[:, :-1], bootstrapped_data[:, -1]
            probas = boost.predict_proba(x)[:, 1]
            
            trues[i] = y.tolist()
            probs[i] = probas.tolist()
    
    return trues, probs


def bootstrap_session(N, models, data_set, device, criterion, show_results=True, method="torch", loc=None):
    
    trues, probs = bootstrap(N, models, data_set, device, criterion, method=method)
    
    if show_results:
        print("Results:")
        calculate_results(N, trues, probs, 0.5) 
    
    if loc is not None:
        results = dict()
        results["true"] = trues
        results["prob"] = probs
        with open(loc, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
    else:
        return trues, probs

In [8]:
N=1000

FM_PATH = MODELS_PATH + r"\FundusModel.pth"
DM_PATH = MODELS_PATH + r"\DemographicFCNN.pth"

criterion = nn.BCEWithLogitsLoss()

# FundusModel Results

In [9]:
image_model = get_densenet201(device=device, freeze=True, with_mlp=True)
image_model.load_state_dict(torch.load(FM_PATH))

In [10]:
models = {
    "image_model": image_model,
    "tabular_model": None,
    "fusion_model": None
}

In [11]:
print("***FundusModel***")
result_path = 'Results/FundusModel.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc=result_path)

***FundusModel***
f1: [0.677, 0.789] average: 0.733
auroc: [0.705, 0.828] average: 0.766
auprc: [0.641, 0.824] average: 0.732
precision: [0.54, 0.687] average: 0.613
recall: [0.864, 0.965] average: 0.914
specificity: [0.333, 0.518] average: 0.426


# DemographicModels Results

In [12]:
train_x, train_y = train_set[["Age", "Gender"]], train_set["Hypertension"]
test_x, test_y = test_set[["Age", "Gender"]], test_set["Hypertension"]

combined_test = np.hstack([test_x, np.expand_dims(test_y, -1)])

## XGBoost

In [13]:
with open(MODELS_PATH+'/DemographicXGBParams.json') as f:
    params = json.load(f)
    
boost = xgb.XGBClassifier(**params, objective="binary:logistic")

boost.fit(train_x, train_y)

In [14]:
print("***DemographicXGB***")
result_path = 'Results/DemographicXGB.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=boost, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc=result_path)

***DemographicXGB***
f1: [0.677, 0.796] average: 0.736
auroc: [0.679, 0.804] average: 0.742
auprc: [0.531, 0.765] average: 0.648
precision: [0.565, 0.718] average: 0.642
recall: [0.807, 0.926] average: 0.866
specificity: [0.426, 0.614] average: 0.52


## SVM

In [15]:
with open(MODELS_PATH+'/DemographicSVMParams.json') as f:
    params = json.load(f)
    
svm = SVC(**params)

svm.fit(train_x, train_y)

In [16]:
print("***DemographicSVM***")
result_path = 'Results/DemographicSVM.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=svm, data_set=combined_test, device=device, criterion=criterion, \
                  method="sklearn", loc=result_path)

***DemographicSVM***
f1: [0.674, 0.79] average: 0.732
auroc: [0.674, 0.802] average: 0.738
auprc: [0.525, 0.765] average: 0.645
precision: [0.561, 0.715] average: 0.638
recall: [0.804, 0.925] average: 0.864
specificity: [0.431, 0.607] average: 0.519


## FCNN

In [17]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

tabular_model = tabular_model.to(device)
tabular_model.load_state_dict(torch.load(DM_PATH))

In [18]:
models = {
    "image_model": None,
    "tabular_model": tabular_model,
    "fusion_model": None
}

In [19]:
print("***DemographicFCNN***")
result_path = 'Results/DemographicFCNN.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc=result_path)

***DemographicFCNN***
f1: [0.677, 0.791] average: 0.734
auroc: [0.699, 0.819] average: 0.759
auprc: [0.65, 0.811] average: 0.73
precision: [0.555, 0.698] average: 0.627
recall: [0.832, 0.946] average: 0.889
specificity: [0.388, 0.571] average: 0.48


# FusionModel Results

## CombineFeatures

In [20]:
tm_path = MODELS_PATH + r"\CombineFeatures\DemographicPath.pth"
img_path = MODELS_PATH + r"\CombineFeatures\FundusPath.pth"
fm_path = MODELS_PATH + r"\CombineFeatures\FusionPath.pth"


tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
)

image_model = get_densenet201(device=device, freeze=True, with_mlp=False, outputs=32)

fusion_model = nn.Sequential(
    nn.Linear(in_features=64, out_features=128),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

tabular_model, image_model, fusion_model = tabular_model.to(device).float(), image_model.to(device).float(),\
                                                fusion_model.to(device).float()

tabular_model.load_state_dict(torch.load(tm_path))
image_model.load_state_dict(torch.load(img_path))
fusion_model.load_state_dict(torch.load(fm_path))

In [21]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

In [22]:
print("***CombineFeatures***")
result_path = 'Results/CombineFeatures.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc=result_path)

***CombineFeatures***
f1: [0.654, 0.775] average: 0.714
auroc: [0.735, 0.848] average: 0.791
auprc: [0.694, 0.851] average: 0.772
precision: [0.532, 0.681] average: 0.606
recall: [0.815, 0.935] average: 0.875
specificity: [0.35, 0.535] average: 0.443


## CombineOutputs

In [23]:
tm_path = MODELS_PATH + r"\CombineOutputs\DemographicPath.pth"
img_path = MODELS_PATH + r"\CombineOutputs\FundusPath.pth"
fm_path = MODELS_PATH + r"\CombineOutputs\FusionPath.pth"

tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=1)

fusion_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=4),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4, out_features=1),
)

tabular_model, image_model, fusion_model = tabular_model.to(device).float(), image_model.to(device).float(),\
                                                fusion_model.to(device).float()

tabular_model.load_state_dict(torch.load(tm_path))
image_model.load_state_dict(torch.load(img_path))
fusion_model.load_state_dict(torch.load(fm_path))

In [24]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

In [25]:
print("***CombineOutputs***")
result_path = 'Results/CombineOutputs.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=models, data_set=test_dataset, device=device, criterion=criterion, loc=result_path)

***CombineOutputs***
f1: [0.655, 0.763] average: 0.709
auroc: [0.622, 0.716] average: 0.669
auprc: [0.749, 0.824] average: 0.787
precision: [0.493, 0.623] average: 0.558
recall: [0.953, 1.0] average: 0.976
specificity: [0.164, 0.309] average: 0.237


## KeepFeatures

In [26]:
image_model = get_densenet201(device=device, freeze=True, with_mlp=True)
image_model.load_state_dict(torch.load(FM_PATH))

image_model = image_model.to(device).float()

models = {
    "image_model": image_model,
    "tabular_model": None,
    "fusion_model": None,
}

In [27]:
train_x, train_y = build_tabular_dataset(models, train_dataset, method="keep_features")

test_x, test_y = build_tabular_dataset(models, test_dataset, method="keep_features")
test_fusion_set = InputOutputDataset(test_x, test_y)

combined_test = np.hstack([test_x, np.expand_dims(test_y, -1)])

## XGBoost

In [28]:
with open(MODELS_PATH+ '/KeepFeatures/XGBParams.json') as f:
    params = json.load(f)
    
boost = xgb.XGBClassifier(**params, objective="binary:logistic")

boost.fit(train_x, train_y)

In [29]:
print("***KeepFeaturesXGB***")
result_path = 'Results/KeepFeaturesXGB.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=boost, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc=result_path)

***KeepFeaturesXGB***
f1: [0.667, 0.79] average: 0.728
auroc: [0.677, 0.8] average: 0.739
auprc: [0.523, 0.772] average: 0.647
precision: [0.556, 0.72] average: 0.638
recall: [0.792, 0.92] average: 0.856
specificity: [0.42, 0.602] average: 0.511


## SVM

In [30]:
with open(MODELS_PATH+ '/KeepFeatures/SVMParams.json') as f:
    params = json.load(f)
        
svm = SVC(**params)

svm.fit(train_x, train_y)

In [31]:
print("***KeepFeaturesSVM***")
result_path = 'Results/KeepFeaturesSVM.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=svm, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc=result_path)

***KeepFeaturesSVM***
f1: [0.669, 0.787] average: 0.728
auroc: [0.677, 0.803] average: 0.74
auprc: [0.526, 0.771] average: 0.649
precision: [0.558, 0.712] average: 0.635
recall: [0.792, 0.92] average: 0.856
specificity: [0.43, 0.608] average: 0.519


## FCNN

In [32]:
fm_path = MODELS_PATH + r"\KeepFeatures\FusionPath.pth"

fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()
fusion_model.load_state_dict(torch.load(fm_path))

In [33]:
models = {
    "image_model": None,
    "tabular_model": None,
    "fusion_model": fusion_model
}

In [34]:
print("***KeepFeaturesFCNN***")
result_path = 'Results/KeepFeaturesFCNN.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=models, data_set=test_fusion_set, device=device, criterion=criterion, loc=result_path)

***KeepFeaturesFCNN***
f1: [0.667, 0.78] average: 0.723
auroc: [0.688, 0.809] average: 0.748
auprc: [0.62, 0.79] average: 0.705
precision: [0.549, 0.697] average: 0.623
recall: [0.807, 0.928] average: 0.868
specificity: [0.404, 0.575] average: 0.49


# VotingFeatures

In [35]:
# Load image and Tabular model
image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=1)

tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

image_model.load_state_dict(torch.load(FM_PATH))
tabular_model.load_state_dict(torch.load(DM_PATH))

image_model, tabular_model = image_model.to(device).float(), tabular_model.to(device).float()

# load fusion model
tabular_path = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
)

image_path = get_densenet201(device=device, freeze=True, with_mlp=False, outputs=32)

fusion_path = nn.Sequential(
    nn.Linear(in_features=64, out_features=128),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

image_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\FundusPath.pth"))
tabular_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\DemographicPath.pth"))
fusion_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\FusionPath.pth"))

tabular_path, image_path, fusion_path = tabular_path.to(device).float(), image_path.to(device).float(),\
                                                fusion_path.to(device).float()

fusion_model = dict()
fusion_model["image_path"], fusion_model["tabular_path"], fusion_model["fusion_path"] = image_path, \
                                                                                    tabular_path, fusion_path

In [36]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

In [37]:
train_x, train_y = build_tabular_dataset(models, train_dataset, method="voting_features")

test_x, test_y = build_tabular_dataset(models, test_dataset, method="voting_features")
test_fusion_set = InputOutputDataset(test_x, test_y)

combined_test = np.hstack([test_x, np.expand_dims(test_y, -1)])

## XGBoost

In [38]:
with open(MODELS_PATH+'/VotingFeatures/XGBParams.json') as f:
    params = json.load(f)
    
boost = xgb.XGBClassifier(**params, objective="binary:logistic")

boost.fit(train_x, train_y)

In [39]:
print("***VotingFeaturesXGB***")
result_path = 'Results/VotingFeaturesXGB.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=boost, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc=result_path)

***VotingFeaturesXGB***
f1: [0.645, 0.769] average: 0.707
auroc: [0.664, 0.791] average: 0.727
auprc: [0.617, 0.797] average: 0.707
precision: [0.531, 0.68] average: 0.606
recall: [0.782, 0.911] average: 0.846
specificity: [0.373, 0.545] average: 0.459


## SVM

In [40]:
with open(MODELS_PATH+ '/KeepFeatures/SVMParams.json') as f:
    params = json.load(f)
        
svm = SVC(**params)

svm.fit(train_x, train_y)

In [41]:
print("***VotingFeaturesSVM***")
result_path = 'Results/VotingFeaturesSVM.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=svm, data_set=combined_test, device=device, criterion=criterion,\
                  method="sklearn", loc=result_path)

***VotingFeaturesSVM***
f1: [0.64, 0.77] average: 0.705
auroc: [0.662, 0.792] average: 0.727
auprc: [0.625, 0.799] average: 0.712
precision: [0.53, 0.693] average: 0.612
recall: [0.775, 0.915] average: 0.845
specificity: [0.372, 0.557] average: 0.464


## FCNN

In [42]:
fm_path = MODELS_PATH + "\VotingFeatures\FusionPath.pth"

fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()
fusion_model.load_state_dict(torch.load(fm_path))

In [43]:
models = {
    "image_model": None,
    "tabular_model": None,
    "fusion_model": fusion_model
}

In [44]:
print("***VotingFeaturesFCNN***")
result_path = 'Results/VotingFeaturesFCNN.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    bootstrap_session(N, models=models, data_set=test_fusion_set, device=device, criterion=criterion, loc=result_path)

***VotingFeaturesFCNN***
f1: [0.664, 0.776] average: 0.72
auroc: [0.682, 0.815] average: 0.749
auprc: [0.619, 0.798] average: 0.708
precision: [0.525, 0.671] average: 0.598
recall: [0.861, 0.962] average: 0.911
specificity: [0.316, 0.5] average: 0.408


## Ensemble

In [45]:
sig = nn.Sigmoid()

tensored_x, tensored_y = torch.tensor(test_x), torch.tensor(test_y)
probs = sig(tensored_x)

average = torch.mean(probs, axis=1)

esnemble_average = torch.stack([average, tensored_y], axis=-1).numpy()

In [46]:
print("***VotingFeaturesEnsemble***")
result_path = 'Results/VotingFeaturesEnsemble.json'

if Path(os.getcwd() + "/" + result_path).is_file():
    f = open(os.getcwd() + "/" + result_path)
    data_json = json.load(f) 
    for y in data_json:
        data_json[y] = {int(k):[i for i in v] for k,v in data_json[y].items()}
    trues, probs = data_json['true'], data_json['prob'] 
    calculate_results(N, trues, probs)
else:
    trues, probs = dict(), dict()
    results = dict()

    for i in tqdm(range(N)):
        bootstrapped_data = resample(esnemble_average, replace=True, n_samples=len(esnemble_average))
        x, y = bootstrapped_data[:, :-1], bootstrapped_data[:, -1]

        trues[i] = y.tolist()
        probs[i] = x.tolist()

    results["true"] = trues
    results["prob"] = probs
    
    with open('Results/VotingFeaturesEnsemble.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    
    trues, probs = [], []

    for i in range(N):
        trues += results["true"][i]
        probs += results["prob"][i]

    calculate_results(N, results['true'], results['prob'], 0.5) 

***VotingFeaturesEnsemble***
f1: [0.661, 0.783] average: 0.722
auroc: [0.649, 0.78] average: 0.715
auprc: [0.557, 0.753] average: 0.655
precision: [0.528, 0.679] average: 0.603
recall: [0.847, 0.953] average: 0.9
specificity: [0.333, 0.516] average: 0.425
